In [1]:
from math import floor
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def load_image(image_path):
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        return np.array(img)

def show_image(before_process_img, after_process_img):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    axes[0].imshow(before_process_img, cmap="gray")
    axes[1].imshow(after_process_img, cmap="gray")
    axes[0].axis("off")
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()

def save_image(after_process_img):
    img = Image.fromarray(after_process_img,mode="RGB")
    img.save("result.jpg")

def bilinear_interpolation(raw_pic : np.ndarray,mul_width,mul_height):
    h, w = raw_pic.shape
    new_h,new_w = h * mul_height,w * mul_width
    scale_h = (h - 1) / (new_h - 1)
    scale_w = (w - 1) / (new_w - 1)
    result = np.zeros((new_h,new_w))
    for i in range(new_h):
        for j in range(new_w):
            y0, x0 = floor(scale_h * i),floor(scale_w * j)
            y1, x1 = min(y0 + 1,h - 1),min(x0 + 1,w - 1)
            dy = scale_h * i - y0
            dx = scale_w * j - x0
            w11,w01,w10,w00 = dy * dx,(1 - dy) * dx,dy * (1 - dx),(1 - dy) * (1 - dx)
            result[i,j] = w11 * raw_pic[y1, x1] + w01 * raw_pic[y0, x1] + w10 * raw_pic[y1, x0] + w00 * raw_pic[y0, x0]
    return result


arr = load_image("me2.jpg")
channels = [arr[:,:,0],arr[:,:,1],arr[:,:,2]]
result = []
for ch in channels:
    result.append(bilinear_interpolation(ch,2,2))
result = np.dstack(result)
result = np.clip(result, 0, 255).astype(np.uint8)
save_image(result)